# Social DistilBERT Model Using Single Batch 1 + Batch 2

In [32]:
import ktrain
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from ktrain import text
import random
import warnings
from sklearn.utils import shuffle

# Set random seed"
random.seed(18)
seed = 18

# Ignore warnings
warnings.filterwarnings('ignore')

# Display options
pd.set_option('display.max_colwidth', None)

## 1. Loading the data and quick exploratory data analysis

In [2]:
# import os

# # Define the folder path and themes
# folder_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/single_theme_using_jaccard_method'
# themes = [
#     'Aspirational', 'Attainment', 'Community Consciousness', 'Familial', 'Filial Piety', 
#     'First Gen', 'Navigational', 'Perseverance', 'Resistance', 'Social', 'Spiritual'
# ]

# # Initialize an empty dictionary to store DataFrames
# batch_1_theme_dataframes = {}
# # Loop through each theme and load its corresponding file
# for theme in themes:
#     # Construct the filename without modifying the theme name
#     file_name = f"{theme}_sentence_level_batch_1_jaccard.csv"
#     file_path = os.path.join(folder_path, file_name)
    
#     # Check if the file exists before attempting to load
#     if os.path.exists(file_path):
#         batch_1_theme_dataframes[theme] = pd.read_csv(file_path)
#         print(f"Loaded {file_name}")
#     else:
#         print(f"File not found for theme: {theme}")

# # Define the folder path and themes
# folder_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/single_theme_using_jaccard_method/batch_2'
# themes = [
#     'Aspirational', 'Attainment', 'Community Consciouss', 'Familial', 'Filial Piety', 
#     'First Generation', 'Navigational', 'Perseverance', 'Resistance', 'Social', 'Spiritual'
# ]

# # Initialize an empty dictionary to store DataFrames
# batch_2_theme_dataframes = {}
# # Loop through each theme and load its corresponding file
# for theme in themes:
#     # Construct the filename without modifying the theme name
#     file_name = f"{theme}_sentence_level_batch_2_jaccard.csv"
#     file_path = os.path.join(folder_path, file_name)
    
#     # Check if the file exists before attempting to load
#     if os.path.exists(file_path):
#         batch_2_theme_dataframes[theme] = pd.read_csv(file_path)
#         print(f"Loaded {file_name}")
#     else:
#         print(f"File not found for theme: {theme}")

In [36]:
merged_social_df = pd.read_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/corrections/Social/Social_merged_Reevaluated_3.csv")


merged_social_df

,sentence,label,phrase,orig_label
0,"i am here because i want to better myself my family, not only financially but in health.",0,"['I Hope to reach a position in which I can inspire and prove to all of the ""immigrants"" in the USA that achieving and surpassing struggle is possible.']",0
1,being in this instituion will pave a way for me to become a professional.,0,"['I Hope to reach a position in which I can inspire and prove to all of the ""immigrants"" in the USA that achieving and surpassing struggle is possible.']",0
2,i know that as a child i never thought of education and a career for someone who is undocumented.,0,"['I Hope to reach a position in which I can inspire and prove to all of the ""immigrants"" in the USA that achieving and surpassing struggle is possible.']",0
3,"i hope to reach a position in which i can inspire and prove to all of the ""immigrants"" in the usa that achieving and surpassing struggle is possible.",1,"['I Hope to reach a position in which I can inspire and prove to all of the ""immigrants"" in the USA that achieving and surpassing struggle is possible.']",1
4,"also, statistics have shown that people with higher education and wealth tend to be healthier.",0,"['I Hope to reach a position in which I can inspire and prove to all of the ""immigrants"" in the USA that achieving and surpassing struggle is possible.']",0
...,...,...,...,...
2955,"when i was younger, my mother was diagnosed with a brain tumor and it took months of convincing her to finally get surgery to remove the tumor.",0,"[""It then narrows down to the things that I find joy in participating in and how I utilize my God given gifts. I'm a person of many talents and skills, which is why I believe I was placed here on this universe for multiple purposes. After experiencing those years of watching her suffer, I've developed a love for the medical field and knew I wanted to be apart of changing the lives of others through medicine. I truly believe my huge personality, when combined with my career interests and natural talents, were placed here to spread love throughout those I cross paths with in life. I'm here to share smiles and be a light in the midst of someone's darkness, while assisting in prolonging long and healthy lives.""]",0
2956,"after experiencing those years of watching her suffer, i've developed a love for the medical field and knew i wanted to be apart of changing the lives of others through medicine.",0,"[""It then narrows down to the things that I find joy in participating in and how I utilize my God given gifts. I'm a person of many talents and skills, which is why I believe I was placed here on this universe for multiple purposes. After experiencing those years of watching her suffer, I've developed a love for the medical field and knew I wanted to be apart of changing the lives of others through medicine. I truly believe my huge personality, when combined with my career interests and natural talents, were placed here to spread love throughout those I cross paths with in life. I'm here to share smiles and be a light in the midst of someone's darkness, while assisting in prolonging long and healthy lives.""]",0
2957,"also, i've been granted the gift of imagination which allows me to write mini books, poems, and short stories, along with a long list of other talents.",0,"[""It then narrows down to the things that I find joy in participating in and how I utilize my God given gifts. I'm a person of many talents and skills, which is why I believe I was placed here on this universe for multiple purposes. After experiencing those years of watching her suffer, I've developed a love for the medical field and knew I wanted to be apart of changing the lives of others through medicine. I truly believe my huge personality, when combined with my career interests and natural talents, were placed here to spread love throughout those I cross paths with in life. I'm here to share smiles and be a light in the midst of someone's darkness

In [37]:
# Process duplicates
merged_social_df = (
    merged_social_df.groupby('sentence', as_index=False)  # Group by sentences
    .agg({'label': 'max'})  # If any label is 1, it takes precedence
)
merged_social_df

,sentence,label
0,"""why am i here"".",0
1,"""why am i here?"" i am in san francisco because i wanted to get out of high school a year early, and specifically, l.a., where i grew up.",0
2,"""why am i here?"" i saw this question as why am i in school.",0
3,"""why am i here?"" the obvious reason why am i here is because this class is required to take for my physics class 230.",0
4,"""why am i here?""such an abstract question; why am i here on this earth right now?",0
...,...,...
2955,you meet great instructors that are positive and want everyone to learn and have a good grip on the concept in order to be confident when taking a quiz or exam.,0
2956,you might not even be the best version of yourself. but still we or i continue to act like these smaller things seem to matter.,0
2957,you never really know what is to come in the next day or so and i happened to learn that the hard way.,0
2958,you so know that the time allotted for the professors are not enough to begin with.,0


In [38]:
# Shuffle the merged dataset
merged_social_df = shuffle(merged_social_df, random_state=seed)

# Train-test split 
training_df, test_df = train_test_split(merged_social_df, test_size=0.1, random_state=18, stratify=merged_social_df["label"])


training_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)


In [39]:
print(f"Training dataset shape: {training_df.shape} \nTest dataset shape: {test_df.shape}")
pos_labels = len([n for n in training_df['label'] if n==1])
print("Positive labels present in the dataset : {}  out of {} or {}%".format(pos_labels, len(training_df['label']), (pos_labels/len(training_df['label']))*100))
pos_labels = len([n for n in test_df['label'] if n==1])
print("Positive labels present in the test dataset : {}  out of {} or {}%".format(pos_labels, len(test_df['label']), (pos_labels/len(test_df['label']))*100))

Training dataset shape: (2664, 2) 
Test dataset shape: (296, 2)
Positive labels present in the dataset : 367  out of 2664 or 13.776276276276276%
Positive labels present in the test dataset : 41  out of 296 or 13.85135135135135%


## 2. Experimental Design

In [41]:
MAXLEN = 150

X = training_df['sentence']
y = training_df['label']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 18, stratify=y)


model_name = 'bert-base-uncased'

distillbert_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0,1])
training_set = distillbert_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
validation_set = distillbert_transformer.preprocess_test(X_test.tolist(), y_test.tolist())
distillbert_base_model = distillbert_transformer.get_classifier()

preprocessing train...
language: en
train sequence lengths:
	mean : 21
	95percentile : 40
	99percentile : 56


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 22
	95percentile : 40
	99percentile : 69


In [42]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Define classes and class labels
classes = np.array([0, 1])
class_labels = list(training_df.label)

# Compute class weights

class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=class_labels)

# Print class weights
print(class_weights)

class_weights = dict(zip(classes, class_weights))

[0.57988681 3.62942779]


In [43]:
# Build BERT model
distillbert_learner = ktrain.get_learner(distillbert_base_model, train_data=training_set, val_data=validation_set, batch_size=6)
distillbert_learner.set_weight_decay(0.001)
distillbert_learner.autofit(3.328132762062889e-05, epochs=12, early_stopping=4, class_weight=class_weights)



begin training using triangular learning rate policy with max lr of 3.328132762062889e-05...
Epoch 1/12
400/400 [==============================] - 303s 681ms/step - loss: 0.5143 - accuracy: 0.7718 - val_loss: 0.3362 - val_accuracy: 0.8689
Epoch 2/12
400/400 [==============================] - 273s 670ms/step - loss: 0.2734 - accuracy: 0.8890 - val_loss: 0.2404 - val_accuracy: 0.9064
Epoch 3/12
400/400 [==============================] - 313s 781ms/step - loss: 0.1462 - accuracy: 0.9487 - val_loss: 0.2135 - val_accuracy: 0.9326
Epoch 4/12
400/400 [==============================] - 285s 712ms/step - loss: 0.0797 - accuracy: 0.9716 - val_loss: 0.1946 - val_accuracy: 0.9401
Epoch 5/12
400/400 [==============================] - 240s 580ms/step - loss: 0.0752 - accuracy: 0.9675 - val_loss: 0.1667 - val_accuracy: 0.9288
Epoch 6/12
400/400 [==============================] - 199s 497ms/step - loss: 0.0820 - accuracy: 0.9666 - val_loss: 0.3068 - val_accuracy: 0.9026
Epoch 7/12
400/400 [=========

In [44]:
distillbert_learner.validate(class_names=distillbert_transformer.get_classes())

9/9 [==============================] - 17s 809ms/step
              precision    recall  f1-score   support

           0       0.98      0.97      0.97       230
           1       0.82      0.86      0.84        37

    accuracy                           0.96       267
   macro avg       0.90      0.92      0.91       267
weighted avg       0.96      0.96      0.96       267



array([[223,   7],
       [  5,  32]])

In [45]:
distillbert_learner.model.summary()

Model: "tf_bert_for_sequence_classification_3"


_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_151 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
distillbert_predictor = ktrain.get_predictor(distillbert_learner.model, preproc=distillbert_transformer)

In [47]:
distillbert_test_data = test_df['sentence'].tolist()
distillbert_test_label = test_df['label'].tolist()

In [48]:
y_pred_distillbert = distillbert_predictor.predict(distillbert_test_data)

In [49]:
y_pred_distillbert = [int(x) for x in y_pred_distillbert]

In [51]:
tn, fp, fn, tp = confusion_matrix(distillbert_test_label, y_pred_distillbert).ravel()
print('True Negative: {}, False Positive: {}, False Negative: {}, True Positive: {}'.format(tn, fp, fn, tp))

True Negative: 248, False Positive: 7, False Negative: 13, True Positive: 28


In [52]:
print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.97      0.96       255
           1       0.80      0.68      0.74        41

    accuracy                           0.93       296
   macro avg       0.88      0.83      0.85       296
weighted avg       0.93      0.93      0.93       296
 



In [16]:
training_df.to_csv("social_training.csv", index=False)
test_df.to_csv("social_test.csv", index=False)

In [53]:
test_df_copy = test_df.copy()
test_df_copy["predicted_label"] = y_pred_distillbert
test_df_copy

,sentence,label,predicted_label
0,so far this class has been surprisingly fun and i actually learned a lot that helps me with the class.,0,0
1,"unfortunately, i joined the sci class as we were beginning to go over unit three, so i am hoping there will be someone i can ask for assistance with the other units.",0,0
2,but i don't think that's the case at all.,0,0
3,i decided to enroll in this 1 unit course as a sort of precautionary measure.,0,0
4,"this makes me empathetic, understanding, and more mature.",0,0
...,...,...,...
291,"as for why i am here in san francisco, i moved away to isolate myself from any distractions that would cause me to not reach my highest potential.",0,0
292,"part of my major requires me to finish math classes, and i wanted to get help with that because classes can be challenging and hard to understand sometimes, and its helpful to have assistance from classes like this in order to feel more confident about my math skills and learn more.",0,0
293,"so far, i feel like i have better understanding with the moon phases, and ive also learned the difference between the big and little dipper (finally).",0,0
294,im taking the physics lab because im also enrolled in the physics lecture on mwf. i'm also taking it because in high school i took preap physics and i really enjoyed it but it was kind of difficult for me as well.,0,0


In [31]:
pwd

'/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/notebooks/experiments/exp_4_single/Social'

In [54]:
test_df_copy.to_excel("Social_capital_error_analysis_2.xlsx")

In [24]:
# distillbert_predictor.save('../../model/first_generation_distilbert_base_uncased_model_10102020') # 256 MB

In [21]:
print("AUC roc score for distillbert model: ", roc_auc_score(distillbert_test_label,y_pred_distillbert))

AUC roc score for distillbert model:  0.9306334080717489


In [22]:
from sklearn.metrics import auc, precision_recall_curve

# Replace these with your actual data
# distillbert_test_label: Ground truth labels (list or numpy array)
# y_pred_distillbert: Predicted probabilities (list or numpy array)

# Calculate precision-recall curve
precision, recall, _ = precision_recall_curve(distillbert_test_label, y_pred_distillbert)

# Calculate PR AUC
pr_auc_score = auc(recall, precision)

# Print the result
print("AUC PR score for DistilBERT model: ", pr_auc_score)

AUC PR score for DistilBERT model:  0.7918205047976637
